In [1]:
import pandas as pd
import pyodbc
import sqlalchemy.types
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [2]:
# Define Database Connection for PROD

CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""

In [3]:
# Define Database Connection for ccsdta

CONNAS400_CCSDTA = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=CCSDTA;
UID=SMY;
PWD=SMY;
"""


In [14]:
sql_inv = """
    SELECT
    b.plt,
    y.itmid,
    b.qty,
    y.itmdesc,
    SUBSTR (Altdesc, 15, 1) Class
FROM
    CCSDTA.DCSCIM y,
    CCSDTA.DMFCMAR x,
    CCSDTA.DCSILM b
WHERE
    x.itmid = y.itmid
    AND x.itmid = b.itmid
    AND x.plt = b.plt
    AND b.plt = '09'
    AND qty <> 0
    AND x.COSTID = 'FRZ'
    AND x.plt NOT IN ('53', '54', '55', '56', '59')
"""

In [15]:
# Improved connection with proper error handling and connection management
try:
    dbcnxn1 = pyodbc.connect(CONNAS400_CCSDTA, timeout=30)
    cursor1 = dbcnxn1.cursor()
    start = timer()

    try:
        cursor1.execute(sql_inv)
        result = cursor1.fetchall()
        msg = str(len(result)) + " AS400 Inventory Records Processed From Table"
    except Exception as e:
        msg = 'AS400 Inventory Query Failed: ' + str(e)
        result = []
        print(msg)
    finally:
        print(msg)
        print("AS400 Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")

except pyodbc.Error as e:
    msg = 'AS400 Connection Failed: ' + str(e)
    result = []
    print(msg)
finally:
    if 'dbcnxn1' in locals():
        dbcnxn1.close()


13964 AS400 Inventory Records Processed From Table
AS400 Connect/Query Time = 0.73 sec


#### Build Dataframe for Component Inventory

In [21]:
df_inv = pd.DataFrame.from_records(result)
data_type_dict = {'PLANT': str, 'ITMID': str, 'QTY': int, 'ITMDESC': str , 'CLASS': str}
df_inv.columns = ['PLANT', 'ITMID', 'QTY', 'ITMDESC', 'CLASS']
df_inv = df_inv.dropna()
df_inv = df_inv.astype(data_type_dict)
df_inv = df_inv.convert_dtypes()
df_inv.sample(20)

,PLANT,ITMID,QTY,ITMDESC,CLASS
8943,09,9009051,1350,BF LOX-ON,2
7474,09,6712595,22220,"67-12-5 1/2"" IND LAB HOME DEP @220",5
484,09,M647248206,1800,BULK @100,5
11799,09,676006175,450,FRS 561290 BULK @500,5
6483,09,6336199,962,"CLP - 9/16"" 63-36-1 - 301 301 204 - 5/16",4
12126,09,533040090016,20,VELVAC #022548 9MM W4 NP CLAMP 10PK@100,5
7265,09,67004010405K,100,670040-104 1/2 KIMBALL/MID#50499 10pk MX,5
4425,09,534050080052,1,12MM W5 IDEAL NON-PERF CLP BULK,5
6166,09,6260451,1000,5/16 MC 301-301-302 SZ 004 0.22-0.63,5
6501,09,6388070,96,63-88-0 9/16 WW GRAINGER 5CZC1 (MX),5


In [40]:
def get_inv():
    try:
        dbcnxn1 = pyodbc.connect(CONNAS400_CCSDTA, timeout=30)
        cursor1 = dbcnxn1.cursor()
        start = timer()

        try:
            cursor1.execute(sql_inv)
            result = cursor1.fetchall()
            msg = str(len(result)) + " AS400 Inventory Records Processed From Table"
        except Exception as e:
            msg = 'AS400 Inventory Query Failed: ' + str(e)
            result = []
            print(msg)
        finally:
            print(msg)
            print("AS400 Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")
            return result
    except pyodbc.Error as e:
        msg = 'AS400 Connection Failed: ' + str(e)
        result = []
        print(msg)
    finally:
        if 'dbcnxn1' in locals():
            dbcnxn1.close()

In [41]:
print(get_inv())

AS400 Inventory Query Failed: ('42S02', '[42S02] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL0204 - DCSCIM in SMY type *FILE not found. (-204) (SQLExecDirectW)')
AS400 Inventory Query Failed: ('42S02', '[42S02] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL0204 - DCSCIM in SMY type *FILE not found. (-204) (SQLExecDirectW)')
AS400 Connect/Query Time = 0.003 sec
[]


In [22]:
def get_emps():
    """Get Employees From iSeries AS400"""
    try:
        dbcnxn = pyodbc.connect(CONNAS400)
        cursor = dbcnxn.cursor()
    except pyodbc.Error as e:
        msg = 'AS400 Connection Failed: ' + str(e)
        print(msg)

    eng_login = ['9999', 'ELMER J FUDD', 'ENG']
    lead_login = ['1208', 'Wile E Coyote', 'ENG']
    eng = ['1208', '9107', '1656', '1472', '1626', '1351']

    strsql = """SELECT STRIP(EMP_CLOCK_NUMBER) As Clock,
                    CONCAT(CONCAT(STRIP(EMP_FIRST_NAME), ' '),
                    STRIP(EMP_LAST_NAME)) As Name,
                    STRIP(EMP_POSITION_CODE) As Code
            FROM PROD.FPCLCKPAY
            WHERE (EMP_LOCATION = 09) AND (EMP_LAST_NAME <> 'TEMP') AND (EMP_SHIFT_TYPE = 'A')
            ORDER BY EMP_CLOCK_NUMBER"""
    try:
        cursor.execute(strsql)
        result = cursor.fetchall()
    except Exception as e:
        msg = 'AS400 Employee Query Failed: ' + str(e)
        result = []
        print(msg)
    else:
        msg = str(len(result)) + " AS400 Employee Records Processed From Table"
        print(msg)
    result.append(eng_login)
    result.append(lead_login)
    for row in result:
        if row[0] in eng:
            row[2] = 'ENG'
    dbcnxn.close()
    return result

In [23]:
print(get_emps())


471 AS400 Employee Records Processed From Table
[('1001', 'DEBBIE POMEROY-GAMMONS', 'DEF'), ('1013', 'BETH THOMPSON', 'DEF'), ('1032', 'THOMAS BOGLE', 'DEF'), ('1039', 'VICKIE ANDERSON', 'DEF'), ('1090', 'BILLY NIPPERS', 'DEF'), ('1122', 'SHELLEY HOUSTON', 'DEF'), ('1139', 'ANTIONETTE BASS', 'DEF'), ('1175', 'WYOMIA CLARK', 'DEF'), ('1176', 'NALEE NHOTSAVANG', 'DEF'), ('1177', 'CHARLES TOOMBS', 'DEF'), ('1213', 'DOUGLAS SHIPP', 'DEF'), ('1238', 'MARICHU RIVERA', 'DEF'), ('1258', 'PHOUVEUNE LAMNGEUN', 'DEF'), ('1318', 'TERESA HOLLIS', 'DEF'), ('1331', 'NANCY CLARK', 'DEF'), ('1332', 'BRENDA CUNNINGHAM', 'DEF'), ('1347', 'HERBERT GRAHAM', 'DEF'), ('1352', 'DALE GERHART', 'DEF'), ('1372', 'SITHONH BOUAPHACHANH', 'DEF'), ('1429', 'GARY BELCHER', 'DEF'), ('1444', 'RILEY BREWER  JR.', 'DEF'), ('1464', 'CHRISTOPHER HIX', 'DEF'), ('1483', 'BARRY KIRBY', 'DEF'), ('1496', 'TERRY MARTIN', 'DEF'), ('1499', 'ACKHASONE PHILAVARN', 'DEF'), ('1514', 'KHAMIN VONGPANYA', 'DEF'), ('1515', 'PHILLIP STINSO